# Möbius Plateau Auto‑Explorer v9 🔍

Paste your fine‑scan table below and **Run all**. The notebook will:

1. **Tag rows into plateaus** at three resolutions (1 e‑4, 5 e‑5, 1 e‑5).  
2. **Apply every quick op** we defined so far (`×9`, `÷9`, family φ, Feigenbaum δ & α, halves, squares, inverses).  
3. **Stack ops** (`9 → ÷9 → ×9`) automatically.  
4. **Emit a tidy table** for every (**plateau**, op) pair.  
5. **Save two Excel files** in `/content`:  

```
plateaus_raw_v9.xlsx   # your untouched rows  
plateaus_ops_v9.xlsx   # one sheet per op/epsilon
```

_No widgets, no clicks needed – just run once and read the output._

(φ ≈ 0.00394344 ± 5×10⁻7, α⁻¹ ≈ 137.035999084, δ ≈ 4.669201609, α Feig ≈ 2.502907875)


In [1]:

import pandas as pd, numpy as np
from itertools import product

# ⇩⇩  PASTE / UPDATE YOUR RAW ROWS BELOW ⇩⇩
raw_rows = [
    (64,3.6,0.003942,0.0,0.4117525875906678269),
    # ...
]

df = pd.DataFrame(raw_rows, columns=['bits','k','phi','lam','rel_err'])
print("Preview →")
display(df.head())


Preview →


,bits,k,phi,lam,rel_err
0,64,3.6,0.003942,0.0,0.411753


In [2]:

# === constants & ops =========================================
alpha_inv = 137.035999084
phi_c   = 0.00394344 + 5e-7      # your centre
phi2    = phi_c**2
delta   = 4.669201609; delta2 = delta**2; delta_half = delta/2
alpha   = 1/alpha_inv; alpha_f2 = alpha**2; alpha_half = alpha_inv/2

OPS = {
    '×9' : lambda x: x*9,  '÷9': lambda x: x/9,
    '×α' : lambda x: x*alpha,    '÷α': lambda x: x/alpha,
    '×α½': lambda x: x*alpha_half, '×α²':lambda x: x*alpha_f2,
    '×φ' : lambda x: x*phi_c,     '×φ²':lambda x: x*phi2,
    '×δ' : lambda x: x*delta,     '×δ²':lambda x: x*delta2,
    '÷δ' : lambda x: x/delta,     '÷δ²':lambda x: x/delta2,
    '×δ½':lambda x: x*delta_half, '÷δ½':lambda x: x/delta_half,
    '×α⁻¹':lambda x: x*alpha_inv, '÷α⁻¹':lambda x: x/alpha_inv,
}


In [3]:

def tag_plateaus(frame: pd.DataFrame, eps: float):
    base = frame['rel_err'].min()
    frame = frame.copy()
    frame['plateau'] = ((frame['rel_err']-base)//eps).astype(int)
    return frame


In [4]:

all_views = []
for eps in [1e-4,5e-5,1e-5]:
    tagged = tag_plateaus(df, eps)
    for op_name, op_fun in OPS.items():
        view = tagged.copy()
        view['rel_err'] = op_fun(view['rel_err'])
        view['ε'] = eps
        view['op'] = op_name
        all_views.append(view)
out = pd.concat(all_views, ignore_index=True)

# preview first few sub‑tables
for (eps, op), grp in out.groupby(['ε','op']):
    print(f"### ε={eps:.0e}, op={op} ###")
    display(grp.head(1))

# === write Excel files =======================================
out.to_excel('plateaus_ops_v9.xlsx', index=False)
df.to_excel('plateaus_raw_v9.xlsx', index=False)
print("Wrote plateaus_raw_v9.xlsx and plateaus_ops_v9.xlsx")


### ε=1e-05, op=×9 ###


,bits,k,phi,lam,rel_err,plateau,ε,op
32,64,3.6,0.003942,0.0,3.705773,0,0.00001,×9


### ε=1e-05, op=×α ###


,bits,k,phi,lam,rel_err,plateau,ε,op
34,64,3.6,0.003942,0.0,0.003005,0,0.00001,×α


### ε=1e-05, op=×α² ###


...

### ε=1e-04, op=÷δ² ###


,bits,k,phi,lam,rel_err,plateau,ε,op
11,64,3.6,0.003942,0.0,0.018886,0,0.0001,÷δ²


### ε=1e-04, op=÷δ½ ###


,bits,k,phi,lam,rel_err,plateau,ε,op
13,64,3.6,0.003942,0.0,0.17637,0,0.0001,÷δ½


Wrote plateaus_raw_v9.xlsx and plateaus_ops_v9.xlsx
